<a href="https://colab.research.google.com/github/sjeena0722/Socar_Hackathon/blob/main/Clustering_ANOVA_Post_Hoc_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 데이터 불러오기
import pandas as pd
df = pd.read_csv('path')
df.head()

In [ ]:
df.groupby('cluster_id')['use_total_hr_mm'].mean()

cluster_id
0    0.322849
1    0.321705
2    0.319741
3    0.324689
4    0.315799
Name: use_total_hr_mm, dtype: float64

In [ ]:
df.groupby('cluster_id')['use_total_hr'].mean()

cluster_id
0     9.511638
1     8.912495
2     7.884372
3    10.474840
4     5.820692
Name: use_total_hr, dtype: float64

In [ ]:
# cluster_id별 count
df.groupby('cluster_id').count()

,age_group,gender,car_model,region,use_total_hr_mm
cluster_id,,,,,
0,110641,110641,110641,110641,110641
1,52272,52272,52272,52272,52272
2,24327,24327,24327,24327,24327
3,72278,72278,72278,72278,72278
4,197549,197549,197549,197549,197549


##  분산분석

In [ ]:
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm

model = ols('use_total_hr_mm ~ C(cluster_id)', df).fit()
anova_lm(model)

,df,sum_sq,mean_sq,F,PR(>F)
C(cluster_id),4.0,6.108985,1.527246,2556.398161,0.0
Residual,457062.0,273.058470,0.000597,NaN,NaN


* Pr(>F)가 p-value. 이 값이 0.05보다 작으면 통계적으로 유의미한 차이가 있음.
* 위의 예시에서는 0.0로 0.05보다 작음. 따라서 유의미한 차이.
* 구체적으로 어떤 수준(집단)이 차이가 있는지 확인하려면 사후분석(post hoc tests)

## 사후분석
- ANOVA 검증 결과 유의미하다는 결론을 얻었을 때, 구체적으로 어떤 수준(들)에서 평균 차이가 나는지를 검증하는 방법


In [ ]:
# 사후분석을 위한 준비
from statsmodels.sandbox.stats.multicomp import MultiComparison
import scipy.stats

comp = MultiComparison(df.use_total_hr_mm, df.cluster_id)


### 1. 봉페로니 교정

In [ ]:
result = comp.allpairtest(scipy.stats.ttest_ind, method='bonf')
result[0]

group1,group2,stat,pval,pval_corr,reject
0,1,7.8991,0.0,0.0,True
0,2,15.5814,0.0,0.0,True
0,3,-12.6594,0.0,0.0,True
0,4,84.5494,0.0,0.0,True
1,2,10.5252,0.0,0.0,True
1,3,-17.8107,0.0,0.0,True
1,4,63.5662,0.0,0.0,True
2,3,-21.6575,0.0,0.0,True
2,4,31.5866,0.0,0.0,True
3,4,90.8754,0.0,0.0,True


* 모든 그룹 간의 평균 차이가 유의미하다. (p < 0.05)

### 2. 투키의 HSD

In [ ]:
from statsmodels.stats.multicomp import pairwise_tukeyhsd

hsd = pairwise_tukeyhsd(df['use_total_hr_mm'], df['cluster_id'], alpha=0.05)
hsd.summary()

group1,group2,meandiff,p-adj,lower,upper,reject
0,1,-0.0011,0.001,-0.0015,-0.0008,True
0,2,-0.0031,0.001,-0.0036,-0.0026,True
0,3,0.0018,0.001,0.0015,0.0022,True
0,4,-0.0071,0.001,-0.0073,-0.0068,True
1,2,-0.002,0.001,-0.0025,-0.0014,True
1,3,0.003,0.001,0.0026,0.0034,True
1,4,-0.0059,0.001,-0.0062,-0.0056,True
2,3,0.0049,0.001,0.0045,0.0054,True
2,4,-0.0039,0.001,-0.0044,-0.0035,True
3,4,-0.0089,0.001,-0.0092,-0.0086,True


* 모든 그룹 간의 평균 차이가 유의미하다. (p < 0.05)

In [ ]:
import numpy as np
# df.groupby('cluster_id')['use_total_hr_mm'].mean()
- np.log(df.groupby('cluster_id')['use_total_hr_mm'].mean())

cluster_id
0    1.130569
1    1.134121
2    1.140244
3    1.124887
4    1.152650
Name: use_total_hr_mm, dtype: float64

In [ ]:
# 0 /2,4 / 1,3

(0.0, 4.0, 3)